In [25]:
from PIL import Image
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils

To create the overlapping splits

In [26]:
def split_image_with_overlap_equal(image_path, overlap=50):
    # Open the image
    img = Image.open(image_path)
    width, height = img.size

    # Calculate the width of each segment so that 3 segments with overlaps exactly span the image width.
    # 3p - 2*overlap = width  ==>  p = (width + 2*overlap) / 3
    p = int((width + 2 * overlap) / 3)

    # Define crop boundaries for each part
    part1_left, part1_right = 0, p
    part2_left, part2_right = p - overlap, (p - overlap) + p
    part3_left, part3_right = 2 * p - 2 * overlap, (2 * p - 2 * overlap) + p

    # Crop each part of the image
    part1 = img.crop((part1_left, 0, part1_right, height))
    part2 = img.crop((part2_left, 0, part2_right, height))
    part3 = img.crop((part3_left, 0, part3_right, height))

    # Create the 'input' directory if it doesn't exist
    output_dir = "input"
    os.makedirs(output_dir, exist_ok=True)

    # Save the resulting image_path in the 'input' directory
    part1.save(os.path.join(output_dir, "part1.jpg"))
    part2.save(os.path.join(output_dir, "part2.jpg"))
    part3.save(os.path.join(output_dir, "part3.jpg"))
    
    print("Image split into three overlapping parts.")

split_image_with_overlap_equal("pano.jpg", overlap=900)


Image split into three overlapping parts.


In [27]:
# Create an ORB extractor
feature_extractor = cv2.ORB_create()

Load Image and Extract Features

In [28]:
def load_and_extract_features(image_file):
    """Load an image and extract ORB keypoints and descriptors."""
    img = cv2.imread(image_file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    keypoints, descriptors = feature_extractor.detectAndCompute(gray, None)
    return img, keypoints, descriptors

Stitch Two Images Together

In [29]:
def stitch_pair(img1, img2, kp1, kp2, desc1, desc2):
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING) # Create a Brute Force matcher
    knn_matches = matcher.knnMatch(desc1, desc2, k=2) # Find the 2 best matches for each descriptor
    good_matches = [m for m, n in knn_matches if m.distance < 0.75 * n.distance]
    # Check if we found enough good matches
    if len(good_matches) < 4:
        print("Not enough matches to compute homography.")
        return None
    # Extract location of good matches
    pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches])
    # Compute homography
    H, _ = cv2.findHomography(pts2, pts1, cv2.RANSAC, 5.0)
    if H is None:
        print("Homography computation failed.")
        return None
    # Warp img2 to img1's perspective
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    # Compute the corners of the image to be warped
    corners_img2 = np.float32([[0, 0], [w2, 0], [w2, h2], [0, h2]]).reshape(-1, 1, 2)
    warped_corners = cv2.perspectiveTransform(corners_img2, H)
    corners_img1 = np.float32([[0, 0], [w1, 0], [w1, h1], [0, h1]]).reshape(-1, 1, 2)
    all_corners = np.concatenate((corners_img1, warped_corners), axis=0)
    # Compute the bounding box of the merged image
    xmin, ymin = np.int32(all_corners.min(axis=0).ravel() - 0.5)
    xmax, ymax = np.int32(all_corners.max(axis=0).ravel() + 0.5)
    # Compute the translation to be applied to img2
    trans = [-xmin, -ymin]
    T = np.array([[1, 0, trans[0]],
                  [0, 1, trans[1]],
                  [0, 0, 1]])
    # Merge the two images
    merged = cv2.warpPerspective(img2, T.dot(H), (xmax - xmin, ymax - ymin))
    merged[trans[1]:trans[1] + h1, trans[0]:trans[0] + w1] = img1
    # Remove black borders by cropping
    gray_merged = cv2.cvtColor(merged, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray_merged, 1, 255, cv2.THRESH_BINARY)
    nonzero = cv2.findNonZero(binary)
    # Compute the bounding box of the non-black region
    x, y, w, h = cv2.boundingRect(nonzero)
    # Crop the image to the bounding box 
    return merged[y:y+h, x:x+w]


Save Annotated Keypoint Images

In [30]:
def save_annotated_keypoints(image_paths, img_list, kp_list, output_dir="output"):
    os.makedirs(output_dir, exist_ok=True)
    # Save the images with annotated keypoints
    for path, img, kps in zip(image_paths, img_list, kp_list):
        annotated = cv2.drawKeypoints(img, kps, None, color=(0, 255, 0))
        out_filename = os.path.join(output_dir, "keypoints_" + os.path.basename(path))
        cv2.imwrite(out_filename, annotated)
        print("Saved annotated image:", out_filename)


Refine the Final Panorama

In [31]:
def crop_panorama(panorama, save_path):
    # Add a black border to the image to avoid errors when finding contours
    bordered = cv2.copyMakeBorder(panorama, 5, 5, 5, 5, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    # Convert the image to grayscale and find contours
    gray = cv2.cvtColor(bordered, cv2.COLOR_BGR2GRAY)
    # Apply a binary threshold to get a black and white image
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    # Find the largest contour
    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Extract the contours and find the largest one
    contours = imutils.grab_contours(contours)
    # Find the largest contour
    largest = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest)
    
    # Create and refine a mask to tightly crop the image
    mask = np.zeros(thresh.shape, dtype="uint8")
    cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)
    refined_mask = mask.copy()
    diff = mask.copy()
    # Erode the mask until the difference is zero
    while cv2.countNonZero(diff) > 0:
        refined_mask = cv2.erode(refined_mask, None)
        diff = cv2.subtract(refined_mask, thresh)
    # Find contours in the refined mask
    contours_refined = cv2.findContours(refined_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_refined = imutils.grab_contours(contours_refined)
    # Find the largest contour in the refined mask
    best = max(contours_refined, key=cv2.contourArea)
    # Crop the image using the bounding box of the largest contour
    X, Y, W, H = cv2.boundingRect(best)
    final_crop = bordered[Y:Y+H, X:X+W]
    # Return the cropped image
    return final_crop

In [32]:
# Get list of .jpg images from the "input" folder
input_dir = "input"
image_paths = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.lower().endswith('.jpg')]

# Load images and extract features
img_list, kp_list, desc_list = [], [], []
for file in image_paths:
    img, kps, desc = load_and_extract_features(file) # Load and extract features
    img_list.append(img) # Append to the list of images
    kp_list.append(kps) # Append to the list of keypoints
    desc_list.append(desc) # Append to the list of descriptors

# Save annotated images with keypoints
save_annotated_keypoints(image_paths, img_list, kp_list)

# Begin stitching with the first two images
panorama = stitch_pair(img_list[0], img_list[1], kp_list[0], kp_list[1], desc_list[0], desc_list[1])
if panorama is None:
    raise Exception("Initial stitching failed.")

# Iteratively stitch remaining images
for i in range(2, len(img_list)):
    # Recompute keypoints on the current panorama for robust merging
    gray_panorama = cv2.cvtColor(panorama, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    new_kps, new_desc = feature_extractor.detectAndCompute(gray_panorama, None)
    # Stitch the new image with the current panorama
    panorama = stitch_pair(panorama, img_list[i], new_kps, kp_list[i], new_desc, desc_list[i])
    if panorama is None:
        print("Stitching failed at image index", i)
        break

# Save the complete stitched panorama
cv2.imwrite("output/stitched_pano.jpg", panorama)

# Refine and crop the final panorama
refined_panorama = crop_panorama(panorama, os.path.join("output", "stitched_pano.jpg"))
print("Refined panorama saved.")


Saved annotated image: output/keypoints_part1.jpg
Saved annotated image: output/keypoints_part2.jpg
Saved annotated image: output/keypoints_part3.jpg
Refined panorama saved.
